In [161]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/creditcardfraud/creditcard.csv


**In this notebook, we have tried to implement linear SVM and apply it on Credit Card Fraud Detection Dataset**

In [236]:
data = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv') # Reading the file .csv
df = pd.DataFrame(data)
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


**In the next step, we have partitioned the data into train and test dataset. The complete dataset has 284807 rows. We have considered training data to be all rows till 100,000th row. Since the data-set here is highly imbalanced, we have employed the technique of under-sampling where we have selected all instances of the rare class and have sampled equal number of instances from the abundant class. In addition, we have made the '0' class as '-1' class.**

In [233]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_train = df[:100000] # We cut in two the original dataset
df_test_all = df[100000:]

df_train_1 = df_train[df_train['Class'] == 1] # We seperate the data which are the frauds and the no frauds
df_train_0 = df_train[df_train['Class'] == 0]
print('Number of Fraud Transaction = ' + str(len(df_train_1)))

df_sample=df_train_0.sample(len(df_train_1))

df_train = pd.concat([df_train_1, df_sample], ignore_index=True)

X_train = df_train.drop(['Time', 'Class'],axis=1)
Y_train = df_train['Class']
X_test = df_test_all.drop(['Time', 'Class'],axis=1)
Y_test = df_test_all['Class']

x_train = X_train.values.tolist()
y_train = Y_train.values.tolist()
x_test = X_test.values.tolist()
y_test = Y_test.values.tolist()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)



for i in range(len(y_train)) :
    if y_train[i] == 0 :
        y_train[i] = -1
for i in range(len(y_test)) :
    if y_test[i] == 0 :
        y_test[i] = -1

print(len(x_train))
print(len(y_train))

Number of Fraud Transaction = 223
446
446


We made the class with 

**Here, we have applied Hinge loss and have tried to optimize the parameters of the model using Gradient Descent algorithm using pytorch module. We tried different values of C and found the best results for C = 0.1**


In [234]:
import torch
import random

C = 0.1

dim = len(x_train[0])
print(dim)
w = torch.autograd.Variable(torch.rand(dim), requires_grad=True)
b = torch.autograd.Variable(torch.rand(1),   requires_grad=True)

step_size = 1e-6
num_epochs = 100
minibatch_size = 20
for epoch in range(num_epochs):
    inds = [i for i in range(len(x_train))]
    random.shuffle(inds)
    for i in range(len(inds)):
        L = C * (max(0, 1 - y_train[inds[i]] * (torch.dot(w, torch.Tensor(x_train[inds[i]])) - b))**2)
        if L != 0: # if the loss is zero, Pytorch leaves the variables as a float 0.0, so we can't call backward() on it
            L.backward()
            w.data -= step_size * w.grad.data # step
            b.data -= step_size * b.grad.data # step
            w.grad.data.zero_()
            b.grad.data.zero_()
print(w,b)

29
tensor([ 0.3942,  0.5590,  0.8403,  0.2795, -0.0033,  0.6035,  0.3497,  0.2059,
         0.0085,  0.2262,  1.0080,  0.5480,  0.8229,  0.2300,  0.1621,  0.5341,
         0.0956,  0.3763,  0.4043,  0.6246,  0.2127,  0.7747,  0.8046,  0.2347,
         0.0692,  0.3383,  0.2353,  0.2641,  0.0104], requires_grad=True) tensor([0.7903], requires_grad=True)


**Then, we tried to assess our algorithm, and the best result we got was around 62%.**

In [237]:
import numpy as np
print('plane equation:  w=', w.detach().numpy(), 'b =', b.detach().numpy()[0])
def accuracy(X, y):
    correct = 0
    for i in range(len(y)):
        y_predicted = int(np.sign((torch.dot(w, torch.Tensor(X[i])) - b).detach().numpy()[0]))
        if y_predicted == y[i]: correct += 1
    return float(correct)/len(y)

print('test accuracy', accuracy(x_test, y_test))

plane equation:  w= [ 0.39423925  0.5590441   0.84026676  0.27949435 -0.00326816  0.6034693
  0.3497401   0.20586573  0.00854138  0.2261779   1.0080268   0.54796916
  0.8228543   0.22997501  0.16206545  0.53409576  0.09564326  0.37628815
  0.40430254  0.624594    0.2127448   0.77473867  0.80459267  0.23470433
  0.06918436  0.3382503   0.2352629   0.26411623  0.01035725] b = 0.79033166
test accuracy 0.6263507334678882


**We tried ensembling methods, since the data-set is highly imbalanced here. We trained 10 Linear SVM models using equal number of positive and negative classes sampled from the training data.**

In [207]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_train = df[:100000] # We cut in two the original dataset
df_test_all = df[100000:]

df_train_1 = df_train[df_train['Class'] == 1] # We seperate the data which are the frauds and the no frauds
df_train_0 = df_train[df_train['Class'] == 0]
print('Number of Fraud Transaction = ' + str(len(df_train_1)))

df_sample_1 =df_train_0.sample(2*len(df_train_1))
df_sample_2 =df_train_0.sample(2*len(df_train_1))
df_sample_3 =df_train_0.sample(2*len(df_train_1))
df_sample_4 =df_train_0.sample(2*len(df_train_1))
df_sample_5 =df_train_0.sample(2*len(df_train_1))
df_sample_6 =df_train_0.sample(2*len(df_train_1))
df_sample_7 =df_train_0.sample(2*len(df_train_1))
df_sample_8 =df_train_0.sample(2*len(df_train_1))
df_sample_9 =df_train_0.sample(2*len(df_train_1))
df_sample_10=df_train_0.sample(2*len(df_train_1))

df_train__1 = pd.concat([df_train_1, df_sample_1], ignore_index=True)
df_train__2 = pd.concat([df_train_1, df_sample_2], ignore_index=True)
df_train__3 = pd.concat([df_train_1, df_sample_3], ignore_index=True)
df_train__4 = pd.concat([df_train_1, df_sample_4], ignore_index=True)
df_train__5 = pd.concat([df_train_1, df_sample_5], ignore_index=True)
df_train__6 = pd.concat([df_train_1, df_sample_6], ignore_index=True)
df_train__7 = pd.concat([df_train_1, df_sample_7], ignore_index=True)
df_train__8 = pd.concat([df_train_1, df_sample_8], ignore_index=True)
df_train__9 = pd.concat([df_train_1, df_sample_9], ignore_index=True)
df_train__10= pd.concat([df_train_1, df_sample_10], ignore_index=True)

X_train__1 = df_train__1.drop(['Time', 'Class'],axis=1)
Y_train__1 = df_train__1['Class']
X_train__2 = df_train__2.drop(['Time', 'Class'],axis=1)
Y_train__2 = df_train__2['Class']
X_train__3 = df_train__3.drop(['Time', 'Class'],axis=1)
Y_train__3 = df_train__3['Class']
X_train__4 = df_train__4.drop(['Time', 'Class'],axis=1)
Y_train__4 = df_train__4['Class']
X_train__5 = df_train__5.drop(['Time', 'Class'],axis=1)
Y_train__5 = df_train__5['Class']
X_train__6 = df_train__6.drop(['Time', 'Class'],axis=1)
Y_train__6 = df_train__6['Class']
X_train__7 = df_train__7.drop(['Time', 'Class'],axis=1)
Y_train__7 = df_train__7['Class']
X_train__8 = df_train__8.drop(['Time', 'Class'],axis=1)
Y_train__8 = df_train__8['Class']
X_train__9 = df_train__9.drop(['Time', 'Class'],axis=1)
Y_train__9 = df_train__9['Class']
X_train__10= df_train__10.drop(['Time', 'Class'],axis=1)
Y_train__10= df_train__10['Class']

X_test = df_test_all.drop(['Time', 'Class'],axis=1)
Y_test = df_test_all['Class']

x_train__1 = X_train__1.values.tolist()
y_train__1 = Y_train__1.values.tolist()
x_train__2 = X_train__2.values.tolist()
y_train__2 = Y_train__2.values.tolist()
x_train__3 = X_train__3.values.tolist()
y_train__3 = Y_train__3.values.tolist()
x_train__4 = X_train__4.values.tolist()
y_train__4 = Y_train__4.values.tolist()
x_train__5 = X_train__5.values.tolist()
y_train__5 = Y_train__5.values.tolist()
x_train__6 = X_train__6.values.tolist()
y_train__6 = Y_train__6.values.tolist()
x_train__7 = X_train__7.values.tolist()
y_train__7 = Y_train__7.values.tolist()
x_train__8 = X_train__8.values.tolist()
y_train__8 = Y_train__8.values.tolist()
x_train__9 = X_train__9.values.tolist()
y_train__9 = Y_train__9.values.tolist()
x_train__10 = X_train__10.values.tolist()
y_train__10 = Y_train__10.values.tolist()

x_train__1 = scaler.fit_transform(x_train__1)
x_train__2 = scaler.fit_transform(x_train__2)
x_train__3 = scaler.fit_transform(x_train__3)
x_train__4 = scaler.fit_transform(x_train__4)
x_train__5 = scaler.fit_transform(x_train__5)
x_train__6 = scaler.fit_transform(x_train__6)
x_train__7 = scaler.fit_transform(x_train__7)
x_train__8 = scaler.fit_transform(x_train__8)
x_train__9 = scaler.fit_transform(x_train__9)
x_train__10 = scaler.fit_transform(x_train__10)

x_test = X_test.values.tolist()
y_test = Y_test.values.tolist()
x_test = scaler.fit_transform(x_test)



for i in range(len(y_train__1)) :
    if y_train__1[i] == 0 :
        y_train__1[i] = -1
for i in range(len(y_train__2)) :
    if y_train__2[i] == 0 :
        y_train__2[i] = -1
for i in range(len(y_train__3)) :
    if y_train__3[i] == 0 :
        y_train__3[i] = -1
for i in range(len(y_train__4)) :
    if y_train__4[i] == 0 :
        y_train__4[i] = -1
for i in range(len(y_train__5)) :
    if y_train__5[i] == 0 :
        y_train__5[i] = -1
for i in range(len(y_train__6)) :
    if y_train__6[i] == 0 :
        y_train__6[i] = -1
for i in range(len(y_train__7)) :
    if y_train__7[i] == 0 :
        y_train__7[i] = -1
for i in range(len(y_train__8)) :
    if y_train__8[i] == 0 :
        y_train__8[i] = -1
for i in range(len(y_train__9)) :
    if y_train__9[i] == 0 :
        y_train__9[i] = -1
for i in range(len(y_train__10)) :
    if y_train__10[i] == 0 :
        y_train__10[i] = -1
for i in range(len(y_test)) :
    if y_test[i] == 0 :
        y_test[i] = -1



Number of Fraud Transaction = 223


In [220]:
import torch
import random

C = 0.1
dim = len(x_train__1[0])
print(dim)
w1 = torch.autograd.Variable(torch.rand(dim), requires_grad=True)
b1 = torch.autograd.Variable(torch.rand(1),   requires_grad=True)

step_size = 1e-6
num_epochs = 500
minibatch_size = 20
for epoch in range(num_epochs):
    inds = [i for i in range(len(x_train__1))]
    random.shuffle(inds)
    for i in range(len(inds)):
        L = C * (max(0, 1 - y_train__1[inds[i]] * (torch.dot(w1, torch.Tensor(x_train__1[inds[i]])) - b1))**2)
        if L != 0: # if the loss is zero, Pytorch leaves the variables as a float 0.0, so we can't call backward() on it
            L.backward()
            w1.data -= step_size * w1.grad.data # step
            b1.data -= step_size * b1.grad.data # step
            w1.grad.data.zero_()
            b1.grad.data.zero_()
print(w1,b1)

dim = len(x_train__2[0])
print(dim)
w2 = torch.autograd.Variable(torch.rand(dim), requires_grad=True)
b2 = torch.autograd.Variable(torch.rand(1),   requires_grad=True)

step_size = 1e-6
num_epochs = 500
minibatch_size = 20
for epoch in range(num_epochs):
    inds = [i for i in range(len(x_train__2))]
    random.shuffle(inds)
    for i in range(len(inds)):
        L = C * (max(0, 1 - y_train__2[inds[i]] * (torch.dot(w2, torch.Tensor(x_train__2[inds[i]])) - b2))**2)
        if L != 0: # if the loss is zero, Pytorch leaves the variables as a float 0.0, so we can't call backward() on it
            L.backward()
            w2.data -= step_size * w2.grad.data # step
            b2.data -= step_size * b2.grad.data # step
            w2.grad.data.zero_()
            b2.grad.data.zero_()
print(w2,b2)



dim = len(x_train__3[0])
print(dim)
w3 = torch.autograd.Variable(torch.rand(dim), requires_grad=True)
b3 = torch.autograd.Variable(torch.rand(1),   requires_grad=True)

step_size = 1e-6
num_epochs = 500
minibatch_size = 20
for epoch in range(num_epochs):
    inds = [i for i in range(len(x_train__3))]
    random.shuffle(inds)
    for i in range(len(inds)):
        L = C * (max(0, 1 - y_train__3[inds[i]] * (torch.dot(w3, torch.Tensor(x_train__3[inds[i]])) - b3))**2)
        if L != 0: # if the loss is zero, Pytorch leaves the variables as a float 0.0, so we can't call backward() on it
            L.backward()
            w3.data -= step_size * w3.grad.data # step
            b3.data -= step_size * b3.grad.data # step
            w3.grad.data.zero_()
            b3.grad.data.zero_()
print(w3,b3)


dim = len(x_train__4[0])
print(dim)
w4 = torch.autograd.Variable(torch.rand(dim), requires_grad=True)
b4 = torch.autograd.Variable(torch.rand(1),   requires_grad=True)

step_size = 1e-6
num_epochs = 500
minibatch_size = 20
for epoch in range(num_epochs):
    inds = [i for i in range(len(x_train__4))]
    random.shuffle(inds)
    for i in range(len(inds)):
        L = C * (max(0, 1 - y_train__4[inds[i]] * (torch.dot(w4, torch.Tensor(x_train__4[inds[i]])) - b4))**2)
        if L != 0: # if the loss is zero, Pytorch leaves the variables as a float 0.0, so we can't call backward() on it
            L.backward()
            w4.data -= step_size * w4.grad.data # step
            b4.data -= step_size * b4.grad.data # step
            w4.grad.data.zero_()
            b4.grad.data.zero_()
print(w4,b4)


dim = len(x_train__5[0])
print(dim)
w5 = torch.autograd.Variable(torch.rand(dim), requires_grad=True)
b5 = torch.autograd.Variable(torch.rand(1),   requires_grad=True)

step_size = 1e-6
num_epochs = 500
minibatch_size = 20
for epoch in range(num_epochs):
    inds = [i for i in range(len(x_train__5))]
    random.shuffle(inds)
    for i in range(len(inds)):
        L = C * (max(0, 1 - y_train__5[inds[i]] * (torch.dot(w5, torch.Tensor(x_train__5[inds[i]])) - b5))**2)
        if L != 0: # if the loss is zero, Pytorch leaves the variables as a float 0.0, so we can't call backward() on it
            L.backward()
            w5.data -= step_size * w5.grad.data # step
            b5.data -= step_size * b5.grad.data # step
            w5.grad.data.zero_()
            b5.grad.data.zero_()
print(w5,b5)


dim = len(x_train__6[0])
print(dim)
w6 = torch.autograd.Variable(torch.rand(dim), requires_grad=True)
b6 = torch.autograd.Variable(torch.rand(1),   requires_grad=True)

step_size = 1e-6
num_epochs = 500
minibatch_size = 20
for epoch in range(num_epochs):
    inds = [i for i in range(len(x_train__6))]
    random.shuffle(inds)
    for i in range(len(inds)):
        L = C * (max(0, 1 - y_train__6[inds[i]] * (torch.dot(w6, torch.Tensor(x_train__6[inds[i]])) - b6))**2)
        if L != 0: # if the loss is zero, Pytorch leaves the variables as a float 0.0, so we can't call backward() on it
            L.backward()
            w6.data -= step_size * w6.grad.data # step
            b6.data -= step_size * b6.grad.data # step
            w6.grad.data.zero_()
            b6.grad.data.zero_()
print(w6,b6)


dim = len(x_train__7[0])
print(dim)
w7 = torch.autograd.Variable(torch.rand(dim), requires_grad=True)
b7 = torch.autograd.Variable(torch.rand(1),   requires_grad=True)

step_size = 1e-6
num_epochs = 500
minibatch_size = 20
for epoch in range(num_epochs):
    inds = [i for i in range(len(x_train__7))]
    random.shuffle(inds)
    for i in range(len(inds)):
        L = C * (max(0, 1 - y_train__7[inds[i]] * (torch.dot(w7, torch.Tensor(x_train__7[inds[i]])) - b7))**2)
        if L != 0: # if the loss is zero, Pytorch leaves the variables as a float 0.0, so we can't call backward() on it
            L.backward()
            w7.data -= step_size * w7.grad.data # step
            b7.data -= step_size * b7.grad.data # step
            w7.grad.data.zero_()
            b7.grad.data.zero_()
print(w7,b7)


dim = len(x_train__8[0])
print(dim)
w8 = torch.autograd.Variable(torch.rand(dim), requires_grad=True)
b8 = torch.autograd.Variable(torch.rand(1),   requires_grad=True)

step_size = 1e-6
num_epochs = 500
minibatch_size = 20
for epoch in range(num_epochs):
    inds = [i for i in range(len(x_train__8))]
    random.shuffle(inds)
    for i in range(len(inds)):
        L = C * (max(0, 1 - y_train__8[inds[i]] * (torch.dot(w8, torch.Tensor(x_train__8[inds[i]])) - b8))**2)
        if L != 0: # if the loss is zero, Pytorch leaves the variables as a float 0.0, so we can't call backward() on it
            L.backward()
            w8.data -= step_size * w8.grad.data # step
            b8.data -= step_size * b8.grad.data # step
            w8.grad.data.zero_()
            b8.grad.data.zero_()
print(w8,b8)


dim = len(x_train__9[0])
print(dim)
w9 = torch.autograd.Variable(torch.rand(dim), requires_grad=True)
b9 = torch.autograd.Variable(torch.rand(1),   requires_grad=True)

step_size = 1e-6
num_epochs = 500
minibatch_size = 20
for epoch in range(num_epochs):
    inds = [i for i in range(len(x_train__9))]
    random.shuffle(inds)
    for i in range(len(inds)):
        L = C * (max(0, 1 - y_train__9[inds[i]] * (torch.dot(w9, torch.Tensor(x_train__9[inds[i]])) - b9))**2)
        if L != 0: # if the loss is zero, Pytorch leaves the variables as a float 0.0, so we can't call backward() on it
            L.backward()
            w9.data -= step_size * w9.grad.data # step
            b9.data -= step_size * b9.grad.data # step
            w9.grad.data.zero_()
            b9.grad.data.zero_()
print(w9,b9)



29
tensor([ 0.0982,  0.5177,  0.4151,  0.5229,  0.1092,  0.2851,  0.1016,  1.0259,
         0.4634, -0.0625,  0.2621,  0.1444,  0.1291,  0.3838,  0.3574,  0.2901,
        -0.0452,  0.2437,  0.4487,  0.0543,  0.1763,  0.7370,  0.5197,  0.0524,
         0.4721,  0.6616,  0.9374,  0.1912,  0.3834], requires_grad=True) tensor([0.4467], requires_grad=True)
29
tensor([ 0.5934,  0.4972,  0.3061,  0.4216,  0.0470,  0.4822,  0.2690,  0.2725,
        -0.1431,  0.4909,  0.3540, -0.1134,  0.3922,  0.6170,  0.3373,  0.4386,
         0.0634, -0.1768,  0.2860,  0.2475,  0.6611,  0.6897,  0.4765,  0.0454,
         0.8471,  0.4324,  0.9845,  0.3309,  0.6906], requires_grad=True) tensor([0.3073], requires_grad=True)
29
tensor([ 0.3610,  0.8172,  0.1382,  0.7256, -0.0893,  0.5799,  0.0748,  0.4162,
         0.3397,  0.6679,  0.7512, -0.0139,  0.6634,  0.6044,  0.3401,  0.7175,
        -0.1508,  0.5561,  0.8890,  0.1139,  0.8930,  0.5154,  0.5048,  0.4396,
         0.5602,  0.2568,  0.2930,  0.4961,  0.75

In [221]:
import numpy as np


print('plane equation:  w=', w.detach().numpy(), 'b =', b.detach().numpy()[0])
def accuracy(X, y):
    correct = 0
    for i in range(len(y)):
        y_predicted = int(np.sign((torch.dot(w1, torch.Tensor(X[i])) - b1).detach().numpy()[0]))
        if y_predicted == y[i]: correct += 1
    return float(correct)/len(y)

print('train accuracy', accuracy(x_train__1, y_train__1))
print('test accuracy', accuracy(x_test, y_test))



print('plane equation:  w=', w.detach().numpy(), 'b =', b.detach().numpy()[0])
def accuracy(X, y):
    correct = 0
    for i in range(len(y)):
        y_predicted = int(np.sign((torch.dot(w2, torch.Tensor(X[i])) - b2).detach().numpy()[0]))
        if y_predicted == y[i]: correct += 1
    return float(correct)/len(y)

print('train accuracy', accuracy(x_train__2, y_train__2))
print('test accuracy', accuracy(x_test, y_test))



print('plane equation:  w=', w.detach().numpy(), 'b =', b.detach().numpy()[0])
def accuracy(X, y):
    correct = 0
    for i in range(len(y)):
        y_predicted = int(np.sign((torch.dot(w3, torch.Tensor(X[i])) - b3).detach().numpy()[0]))
        if y_predicted == y[i]: correct += 1
    return float(correct)/len(y)

print('train accuracy', accuracy(x_train__3, y_train__3))
print('test accuracy', accuracy(x_test, y_test))


print('plane equation:  w=', w.detach().numpy(), 'b =', b.detach().numpy()[0])
def accuracy(X, y):
    correct = 0
    for i in range(len(y)):
        y_predicted = int(np.sign((torch.dot(w4, torch.Tensor(X[i])) - b4).detach().numpy()[0]))
        if y_predicted == y[i]: correct += 1
    return float(correct)/len(y)

print('train accuracy', accuracy(x_train__4, y_train__4))
print('test accuracy', accuracy(x_test, y_test))


print('plane equation:  w=', w.detach().numpy(), 'b =', b.detach().numpy()[0])
def accuracy(X, y):
    correct = 0
    for i in range(len(y)):
        y_predicted = int(np.sign((torch.dot(w5, torch.Tensor(X[i])) - b5).detach().numpy()[0]))
        if y_predicted == y[i]: correct += 1
    return float(correct)/len(y)

print('train accuracy', accuracy(x_train__5, y_train__5))
print('test accuracy', accuracy(x_test, y_test))


print('plane equation:  w=', w.detach().numpy(), 'b =', b.detach().numpy()[0])
def accuracy(X, y):
    correct = 0
    for i in range(len(y)):
        y_predicted = int(np.sign((torch.dot(w6, torch.Tensor(X[i])) - b6).detach().numpy()[0]))
        if y_predicted == y[i]: correct += 1
    return float(correct)/len(y)

print('train accuracy', accuracy(x_train__6, y_train__6))
print('test accuracy', accuracy(x_test, y_test))


print('plane equation:  w=', w.detach().numpy(), 'b =', b.detach().numpy()[0])
def accuracy(X, y):
    correct = 0
    for i in range(len(y)):
        y_predicted = int(np.sign((torch.dot(w7, torch.Tensor(X[i])) - b7).detach().numpy()[0]))
        if y_predicted == y[i]: correct += 1
    return float(correct)/len(y)

print('train accuracy', accuracy(x_train__7, y_train__7))
print('test accuracy', accuracy(x_test, y_test))



print('plane equation:  w=', w.detach().numpy(), 'b =', b.detach().numpy()[0])
def accuracy(X, y):
    correct = 0
    for i in range(len(y)):
        y_predicted = int(np.sign((torch.dot(w8, torch.Tensor(X[i])) - b8).detach().numpy()[0]))
        if y_predicted == y[i]: correct += 1
    return float(correct)/len(y)

print('train accuracy', accuracy(x_train__8, y_train__8))
print('test accuracy', accuracy(x_test, y_test))


print('plane equation:  w=', w.detach().numpy(), 'b =', b.detach().numpy()[0])
def accuracy(X, y):
    correct = 0
    for i in range(len(y)):
        y_predicted = int(np.sign((torch.dot(w9, torch.Tensor(X[i])) - b9).detach().numpy()[0]))
        if y_predicted == y[i]: correct += 1
    return float(correct)/len(y)

print('train accuracy', accuracy(x_train__9, y_train__9))
print('test accuracy', accuracy(x_test, y_test))

plane equation:  w= [ 0.05662426  0.22886802 -0.17517106  0.3631412   0.7576581  -0.10271765
 -0.16986188  0.2745875   0.1871479   0.48516273  0.29159537  0.23220786
  0.09014101  0.67044294  0.88776237  0.4560186   0.6516656   0.18677628
  0.9792622   0.31844077  0.14111124  0.36665717  0.04553364  0.92214006
  0.92088187  0.83406633  0.2608149   0.08971967  0.1773639 ] b = 0.56405
train accuracy 0.6278026905829597
test accuracy 0.5791122630636285
plane equation:  w= [ 0.05662426  0.22886802 -0.17517106  0.3631412   0.7576581  -0.10271765
 -0.16986188  0.2745875   0.1871479   0.48516273  0.29159537  0.23220786
  0.09014101  0.67044294  0.88776237  0.4560186   0.6516656   0.18677628
  0.9792622   0.31844077  0.14111124  0.36665717  0.04553364  0.92214006
  0.92088187  0.83406633  0.2608149   0.08971967  0.1773639 ] b = 0.56405
train accuracy 0.5037369207772795
test accuracy 0.5495841607731309
plane equation:  w= [ 0.05662426  0.22886802 -0.17517106  0.3631412   0.7576581  -0.10271765
 

In [222]:
import numpy as np


print('plane equation:  w=', w.detach().numpy(), 'b =', b.detach().numpy()[0])
def accuracy(X, y):
    correct = 0
    for i in range(len(y)):
        count = 0
        y_predicted_1 = int(np.sign((torch.dot(w1, torch.Tensor(X[i])) - b1).detach().numpy()[0]))
        y_predicted_2 = int(np.sign((torch.dot(w2, torch.Tensor(X[i])) - b2).detach().numpy()[0]))
        y_predicted_3 = int(np.sign((torch.dot(w3, torch.Tensor(X[i])) - b3).detach().numpy()[0]))
        y_predicted_4 = int(np.sign((torch.dot(w4, torch.Tensor(X[i])) - b4).detach().numpy()[0]))
        y_predicted_5 = int(np.sign((torch.dot(w5, torch.Tensor(X[i])) - b5).detach().numpy()[0]))
        y_predicted_6 = int(np.sign((torch.dot(w6, torch.Tensor(X[i])) - b6).detach().numpy()[0]))
        y_predicted_7 = int(np.sign((torch.dot(w6, torch.Tensor(X[i])) - b6).detach().numpy()[0]))
        y_predicted_8 = int(np.sign((torch.dot(w8, torch.Tensor(X[i])) - b8).detach().numpy()[0]))
        y_predicted_9 = int(np.sign((torch.dot(w9, torch.Tensor(X[i])) - b9).detach().numpy()[0]))
        if y_predicted_1 == y[i]: 
            count += 1
        if y_predicted_2 == y[i]: 
            count += 1
        if y_predicted_3 == y[i]: 
            count += 1
        if y_predicted_4 == y[i]: 
            count += 1
        if y_predicted_5 == y[i]: 
            count += 1
        if y_predicted_6 == y[i]: 
            count += 1
        if y_predicted_7 == y[i]: 
            count += 1
        if y_predicted_8 == y[i]: 
            count += 1
        if y_predicted_9 == y[i]: 
            count += 1
        if count > 4 :
            correct += 1
    return float(correct)/len(y)

print('train accuracy', accuracy(x_train__1, y_train__1))
print('train accuracy', accuracy(x_train__2, y_train__2))
print('train accuracy', accuracy(x_train__3, y_train__3))
print('train accuracy', accuracy(x_train__4, y_train__4))
print('train accuracy', accuracy(x_train__5, y_train__5))
print('train accuracy', accuracy(x_train__6, y_train__6))
print('train accuracy', accuracy(x_train__7, y_train__7))
print('train accuracy', accuracy(x_train__8, y_train__8))
print('train accuracy', accuracy(x_train__9, y_train__9))
print('test accuracy', accuracy(x_test, y_test))

plane equation:  w= [ 0.05662426  0.22886802 -0.17517106  0.3631412   0.7576581  -0.10271765
 -0.16986188  0.2745875   0.1871479   0.48516273  0.29159537  0.23220786
  0.09014101  0.67044294  0.88776237  0.4560186   0.6516656   0.18677628
  0.9792622   0.31844077  0.14111124  0.36665717  0.04553364  0.92214006
  0.92088187  0.83406633  0.2608149   0.08971967  0.1773639 ] b = 0.56405
train accuracy 0.617339312406577
train accuracy 0.6158445440956651
train accuracy 0.6188340807174888
train accuracy 0.6083707025411061
train accuracy 0.5979073243647235
train accuracy 0.5964125560538116
train accuracy 0.6098654708520179
train accuracy 0.6233183856502242
train accuracy 0.6188340807174888
test accuracy 0.593153938974173


**The accuracy of the ensembled models was equivalent to the accuracy of single classifier. Ensembling methods didn't seem to improve the performance of our Linear SVM model and the accuracy on the test data-set was still 60%**